# DATASCI W261: Machine Learning at Scale

## This is an example of reducer side inner join

## Write two tables into files

In [1]:
%%writefile customers.dat
1|Alice Bob|31|CA
2|Sam Sneed|51|NV
3|Jon Sneed|37|CA
4|Arnold Wesise|17|NY
5|Henry Bob|25|NV
6|Yo Yo Ma|37|NY
7|Jon York|41|WA
8|Alex Ball|26|WA
9|Jim Davis|19|CA

Writing customers.dat


In [2]:
%%writefile orders.dat
1|Apple
3|Garlic
2|Milk
1|Iphone
4|Ipad
5|Book
7|Potato
8|Tomato
9|Orange
5|shoes

Writing orders.dat


## MrJob class for ReducerSideInnerJoin

In [3]:
%%writefile reducersideinnerjoin.py
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from mrjob.compat import get_jobconf_value
 
class innerjoin(MRJob):
    def mapper(self, _, line):
        x = line.split("|")
        if len(x) == 4:
            yield x[0], ("lefttable", x[1], x[2], x[3])
        else:
            yield x[0], ("righttable", x[1])

    def reducer(self, key, values):
        customers = list()
        orders = list()
        for val in values:
            if val[0] == u'lefttable':
                customers.append(val)
            else:
                orders.append(val)
        for o in orders:
            for c in customers:
                yield None, [key] + c[1:] + o[1:]

if __name__ == '__main__':
    innerjoin.run()

Writing reducersideinnerjoin.py


## Run the code through python driver

####  Reminder: You cannot use the programmatic runner functionality in the same file as your job class. That is because the file with the job class is sent to Hadoop to be run. Therefore, the job file cannot attempt to start the Hadoop job, or you would be recursively creating Hadoop jobs!

Use make_runner() to run an MRJob
1. seperate driver from mapreduce jobs
2. now we can run it within pythonnode book 
3. In python, typically one class is in each file. Each mrjob job is a seperate class, should be in a seperate file

In [4]:
from reducersideinnerjoin import innerjoin
mr_job = innerjoin(args=['customers.dat','orders.dat'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print value
        count = count + 1
print "\n"
print "There are %s records" %count

['1', 'Alice Bob', '31', 'CA', 'Apple']
['1', 'Alice Bob', '31', 'CA', 'Iphone']
['2', 'Sam Sneed', '51', 'NV', 'Milk']
['3', 'Jon Sneed', '37', 'CA', 'Garlic']
['4', 'Arnold Wesise', '17', 'NY', 'Ipad']
['5', 'Henry Bob', '25', 'NV', 'Book']
['5', 'Henry Bob', '25', 'NV', 'shoes']
['7', 'Jon York', '41', 'WA', 'Potato']
['8', 'Alex Ball', '26', 'WA', 'Tomato']
['9', 'Jim Davis', '19', 'CA', 'Orange']


There are 10 records


## Now do the Left join

In [5]:
%%writefile reducersideleftjoin.py
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from mrjob.compat import get_jobconf_value
 
class leftjoin(MRJob):
    def mapper(self, _, line):
        x = line.split("|")
        if len(x) == 4:
            yield x[0], ("lefttable", x[1], x[2], x[3])
        else:
            yield x[0], ("righttable", x[1])

    def reducer(self, key, values):
        customers = list()
        orders = list()
        for val in values:
            if val[0] == u'lefttable':
                customers.append(val)
            else:
                orders.append(val)
        for c in customers:
            if len(orders) > 0:
                for o in orders:
                    yield None, [key] + c[1:] + o[1:]
            else:
                yield None, [key] + c[1:]

if __name__ == '__main__':
    leftjoin.run()

Writing reducersideleftjoin.py


In [6]:
from reducersideleftjoin import leftjoin
mr_job = leftjoin(args=['customers.dat','orders.dat'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print value
        count = count + 1
print "\n"
print "There are %s records" %count

['1', 'Alice Bob', '31', 'CA', 'Apple']
['1', 'Alice Bob', '31', 'CA', 'Iphone']
['2', 'Sam Sneed', '51', 'NV', 'Milk']
['3', 'Jon Sneed', '37', 'CA', 'Garlic']
['4', 'Arnold Wesise', '17', 'NY', 'Ipad']
['5', 'Henry Bob', '25', 'NV', 'Book']
['5', 'Henry Bob', '25', 'NV', 'shoes']
['6', 'Yo Yo Ma', '37', 'NY']
['7', 'Jon York', '41', 'WA', 'Potato']
['8', 'Alex Ball', '26', 'WA', 'Tomato']
['9', 'Jim Davis', '19', 'CA', 'Orange']


There are 11 records


## Now do the right join

In [7]:
%%writefile reducersiderightjoin.py
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from mrjob.compat import get_jobconf_value
 
class rightjoin(MRJob):
    def mapper(self, _, line):
        x = line.split("|")
        if len(x) == 4:
            yield x[0], ("lefttable", x[1], x[2], x[3])
        else:
            yield x[0], ("righttable", x[1])

    def reducer(self, key, values):
        customers = list()
        orders = list()
        for val in values:
            if val[0] == u'lefttable':
                customers.append(val)
            else:
                orders.append(val)
        for o in orders:
            if len(customers) > 0:
                for c in customers:
                    yield None, [key] + c[1:] + o[1:]
            else:
                yield None, [key] + o[1:]

if __name__ == '__main__':
    rightjoin.run()

Writing reducersiderightjoin.py


In [9]:
from reducersiderightjoin import rightjoin
mr_job = rightjoin(args=['customers.dat','orders.dat'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print value
        count = count + 1
print "\n"
print "There are %s records" %count

['1', 'Alice Bob', '31', 'CA', 'Apple']
['1', 'Alice Bob', '31', 'CA', 'Iphone']
['2', 'Sam Sneed', '51', 'NV', 'Milk']
['3', 'Jon Sneed', '37', 'CA', 'Garlic']
['4', 'Arnold Wesise', '17', 'NY', 'Ipad']
['5', 'Henry Bob', '25', 'NV', 'Book']
['5', 'Henry Bob', '25', 'NV', 'shoes']
['7', 'Jon York', '41', 'WA', 'Potato']
['8', 'Alex Ball', '26', 'WA', 'Tomato']
['9', 'Jim Davis', '19', 'CA', 'Orange']


There are 10 records
